<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Augusto Tagle Montes

## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** https://github.com/atagle123/Evaluaciones_MDS7202

# Importamos librerias útiles

# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

C:\Users\augus\AppData\Local\Temp\ipykernel_26596\3184305967.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [2]:
# Inserte su código acá
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,FunctionTransformer
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import pickle

seed=42
object_cols = df.select_dtypes(include=['object']).columns
df[object_cols] = df[object_cols].astype('category')

Y=df.loc[:,"quantity"]
X=df.drop(['quantity',"city","lat","long", 'id'], axis=1)  # se eliminan estas variables ya que apuntan a lo mismo y/o no aportan información

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1,shuffle=True, random_state=seed) # se separan los datos
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.222,shuffle=True, random_state=seed)

def extract_date(df):  # se extrae la fecha 
    try:
        df['day'] = df['date'].dt.day.astype('category')
        df['month'] = df['date'].dt.month.astype('category')
        df['year'] = df['date'].dt.year.astype('category')
        df.drop("date", axis=1, inplace=True)
    except:
        pass
    return(df)

date_preprocessor=FunctionTransformer(extract_date)

numeric_columns = ['pop', 'price']
categorical_columns = ['shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']

preprocessor = ColumnTransformer(transformers=[
        ('num', "passthrough", numeric_columns), # no hace nada con las variable snumericas
        ('cat', OneHotEncoder(sparse_output=False), categorical_columns)])

preprocessor.set_output(transform="pandas")

steps = [
    ('date', date_preprocessor),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor())
]
pipeline = Pipeline(steps)

pipeline.fit(X_train, y_train)
y_pred_val = pipeline.predict(X_val)

mae=mean_absolute_error(y_val, y_pred_val)
print(mae)

13545.190961454393


Esta métrica se interpreta como el la cantidad de unidades promedio de errror que predice el modelo

Xgboost

In [3]:
steps = [
    ('date', date_preprocessor),  
    ('preprocessor', preprocessor),
    ('regressor', xgb.XGBRegressor(seed=seed))
]
pipeline_xgb = Pipeline(steps)

pipeline_xgb.fit(X_train, y_train)
y_pred_val = pipeline_xgb.predict(X_val)

mae=mean_absolute_error(y_val, y_pred_val)
print(mae)

2458.650460262427


Al implementar Xgboost, el MAE se reduce a un 20% aprox del resultado del dummy regressor. Por lo tanto es mejor que el dummy regressor.

In [4]:
model=pipeline  # se guardan los modelos en pkl
with open('model_dummy.pkl', 'wb') as file:
    pickle.dump(model, file)

model=pipeline_xgb
with open('model_xgb_1.pkl', 'wb') as file:
    pickle.dump(model, file)

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [5]:
# Inserte su código acá
monotone_constraints={'num__price':-1}  # se utiliza esto como constraint

numeric_columns = ['pop', 'price']
categorical_columns = ['shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']

preprocessor = ColumnTransformer(transformers=[
        ('num', "passthrough", numeric_columns),
        ('cat', OneHotEncoder(sparse_output=False), categorical_columns)])


preprocessor.set_output(transform="pandas")

steps = [
    ('date', date_preprocessor),
    ('preprocessor', preprocessor),
    ('regressor', xgb.XGBRegressor(monotone_constraints=monotone_constraints,seed=seed))
]
pipeline_xgb_2 = Pipeline(steps)
pipeline_xgb_2
pipeline_xgb_2.fit(X_train, y_train)
y_pred_val = pipeline_xgb_2.predict(X_val)
mae=mean_absolute_error(y_val, y_pred_val)
print(mae)


2580.049121708198


El error sube ligeramente al introducir este constraint, por lo que si bien no es suficientemente grande como para decir que mi amigo estaba equivocado, se puede decir que el modelo anterior sin constraints,  lograba capturar de manera similar la relación entre el precio y la cantidad.

In [6]:
model=pipeline_xgb_2  # se guarda el modelo
with open('model_xgb_2.pkl', 'wb') as file:
    pickle.dump(model, file)

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [7]:
# Inserte su código acá
import tqdm as notebook_tqdm
import optuna
from optuna.visualization import plot_optimization_history
from optuna.samplers import TPESampler

seed=42

def objective_function(trial):
    global pipeline_xgb_3

    params = {  # hiperparametros de XGB
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "max_leaves": trial.suggest_int("max_leaves", 0, 100),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
    }
    params_one_hot={"min_frequency": trial.suggest_float("min_frequency", 0, 1)} # hiperparametros de one hot 

    numeric_columns = ['pop', 'price']
    categorical_columns = ['shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']

    preprocessor = ColumnTransformer(transformers=[
            ('num', "passthrough", numeric_columns),
            ('cat', OneHotEncoder(sparse_output=False,**params_one_hot), categorical_columns)])


    preprocessor.set_output(transform="pandas")
    steps = [
        ('date', date_preprocessor),  
        ('preprocessor', preprocessor),
        ('regressor', xgb.XGBRegressor(**params,seed=seed))
        ]
    pipeline_xgb_3 = Pipeline(steps)

    pipeline_xgb_3.fit(X_train, y_train)
    y_pred_val = pipeline_xgb_3.predict(X_val)
    mae=mean_absolute_error(y_val, y_pred_val)
    return(mae)

best_model = None
pipeline_xgb_3 = None

def callback(study, trial):   # esto para guardar el mejor modelo
    global best_model
    if study.best_trial == trial:
        best_model = pipeline_xgb_3

c:\Users\augus\anaconda3\envs\Lab1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
sampler = TPESampler(seed=seed)  # esto para que los resultados sean reproducibles
study = optuna.create_study(direction="minimize",sampler=sampler)
study.optimize(objective_function, timeout=300,callbacks=[callback])

[I 2023-11-17 15:00:44,395] A new study created in memory with name: no-name-ef6cbe19-9990-48fb-9759-5bf2e1ac2edc


[I 2023-11-17 15:00:45,857] Trial 0 finished with value: 9737.152903471057 and parameters: {'max_depth': 5, 'learning_rate': 0.0951207163345817, 'n_estimators': 746, 'max_leaves': 60, 'min_child_weight': 1, 'reg_alpha': 0.15599452033620265, 'reg_lambda': 0.05808361216819946, 'min_frequency': 0.8661761457749352}. Best is trial 0 with value: 9737.152903471057.
[I 2023-11-17 15:00:46,219] Trial 1 finished with value: 5911.123439466233 and parameters: {'max_depth': 7, 'learning_rate': 0.07109918520180851, 'n_estimators': 69, 'max_leaves': 97, 'min_child_weight': 5, 'reg_alpha': 0.21233911067827616, 'reg_lambda': 0.18182496720710062, 'min_frequency': 0.18340450985343382}. Best is trial 1 with value: 5911.123439466233.
[I 2023-11-17 15:00:47,005] Trial 2 finished with value: 9061.430245910235 and parameters: {'max_depth': 5, 'learning_rate': 0.05295088673159155, 'n_estimators': 460, 'max_leaves': 29, 'min_child_weight': 4, 'reg_alpha': 0.13949386065204183, 'reg_lambda': 0.29214464853521815, 

 El hiperparametro "max_depth" se refiere a la profundidad maxima de cada arbol, y hace sentido buscar entre 3 y 10.  
 El "learning_rate" es la tasa a la que se entrena el modelo y valores entre 0.001, 0.1 hacen sentido, sin embargo se podria ampliar el rango de busqueda.  
"n_estimators" se refiere a la cantidad de arboles que se usan por modelo, usar entre 50 y 1000 parece adecuado.  
"max_leaves" se refiere al la cantidad maxima de hojas por árbol, de 0 a 100 parece adecuado.  
"min_child_weight" es la cantidad minima de nodos que deben salir de otro nodo, para que sea particion. Entre   1 y 5 parece adecuado.  
reg_alpha" Es el termino de regularizacion L1 (Lasso). Entre 0 y 1 es adecuado.  
"reg_lambda" Es el termino de regularizacion L2 (Risge). Entre 0 y 1 es adecuado.  

Por úlitmo "min frequency" en el one hot encoder, es la cantidad minima de veces que tiene que aparecer una categoría, para que sea considerada. Como aca es float, equivale al ratio, por lo que es adecuado buscar entre 0 y 1.

In [9]:
study.best_value  # mejor valor de MAE

1973.655046337563

El valor encontrado aqui es mejor que el encontrado anteriormente, ya que se hizo una busqueda óptima de los hiperparametros, por lo que tiene sentido que sea mejor.

In [10]:
study.best_params  # mejores hiperparametros

{'max_depth': 10,
 'learning_rate': 0.09756382450533208,
 'n_estimators': 705,
 'max_leaves': 78,
 'min_child_weight': 3,
 'reg_alpha': 0.8396024252429328,
 'reg_lambda': 0.6772215909564098,
 'min_frequency': 0.058459412837189}

In [11]:
study.trials_dataframe()  # todos los trials

,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_max_depth,params_max_leaves,params_min_child_weight,params_min_frequency,params_n_estimators,params_reg_alpha,params_reg_lambda,state
0,0,9737.152903,2023-11-17 15:00:44.398783,2023-11-17 15:00:45.854247,0 days 00:00:01.455464,0.095121,5,60,1,0.866176,746,0.155995,0.058084,COMPLETE
1,1,5911.123439,2023-11-17 15:00:45.860250,2023-11-17 15:00:46.217348,0 days 00:00:00.357098,0.071099,7,97,5,0.183405,69,0.212339,0.181825,COMPLETE
2,2,9061.430246,2023-11-17 15:00:46.226712,2023-11-17 15:00:47.004827,0 days 00:00:00.778115,0.052951,5,29,4,0.366362,460,0.139494,0.292145,COMPLETE
3,3,6030.698483,2023-11-17 15:00:47.009895,2023-11-17 15:00:47.759996,0 days 00:00:00.750101,0.078732,6,51,3,0.170524,239,0.046450,0.607545,COMPLETE
4,4,8995.859048,2023-11-17 15:00:47.763998,2023-11-17 15:00:48.712955,0 days 00:00:00.948957,0.094940,3,81,2,0.440152,968,0.097672,0.684233,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,152,2029.592620,2023-11-17 15:05:26.108235,2023-11-17 15:05:30.411164,0 days 00:00:04.302929,0.098139,9,75,3,0.039620,800,0.998526,0.688271,COMPLETE
153,153,2083.347376,2023-11-17 15:05:30.416177,2023-11-17 15:05:34.370516,0 days 00:00:03.954339,0.094278,9,71,3,0.058265,699,0.980057,0.669651,COMPLETE
154,154,2771.230604,2023-11-17 15:05:34.375515,2023-11-17 15:05:36.384430,0 days 00:00:02.008915,0.091122,3,77,3,0.017024,735,0.961712,0.702466,COMPLETE
155,155,6040.195898,2023-11-17 15:05:36.389431,2023-11-17 15:05:39.952156,0 days 00:00:03.562725,0.096495,9,72,3,0.088148,764,0.947084,0.796893,COMPLETE


Hay 157 trials

In [12]:
model=best_model # se guarda el mejor modelo
with open('model_xgb_3.pkl', 'wb') as file:  # se guarda el mejor modelo
    pickle.dump(model, file)

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

**Respuesta 1:**  
El prunning consiste en cortar los trials que no sean relevantes, respecto a la métrica que se entrega. De modo que se agiliza el entrenamiento, y encuentra posibles mejores hiperparametros.

In [13]:
# Inserte su código acá

seed=42

def objective_function(trial):
    global pipeline_xgb_4
    params = {
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "max_leaves": trial.suggest_int("max_leaves", 0, 100),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
    }
    params_one_hot={"min_frequency": trial.suggest_float("min_frequency", 0, 1)}

    numeric_columns = ['pop', 'price']
    categorical_columns = ['shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']

    preprocessor = ColumnTransformer(transformers=[
            ('num', "passthrough", numeric_columns),
            ('cat', OneHotEncoder(sparse_output=False,**params_one_hot), categorical_columns)])

    pruning_callback = optuna.integration.XGBoostPruningCallback(   # prunning callback con metrica de validation mae
    trial,observation_key="validation_0-mae")
    
    preprocessor.set_output(transform="pandas")
    steps = [
        ('date', date_preprocessor),
        ('preprocessor', preprocessor),
        ('regressor', xgb.XGBRegressor(**params,seed=seed,eval_metric="mae"))   # se entrega la metrica de evaluacion y los hiperparametros
        ]
    pipeline_xgb_4 = Pipeline(steps)

    X_val_transformed=pipeline_xgb_4.named_steps['date'].transform(X_val)  # se transforma el x_val para que pueda ser evaluado

    pipeline_xgb_4.named_steps['preprocessor'].fit_transform(X_train)
    X_val_transformed=pipeline_xgb_4.named_steps['preprocessor'].transform(X_val_transformed)
    
    pipeline_xgb_4.fit(X_train, y_train,regressor__callbacks=[pruning_callback],regressor__eval_set=[(X_val_transformed, y_val)])  # se entrega el prunning y el conjunto de evaluación
    y_pred_val = pipeline_xgb_4.predict(X_val)
    mae=mean_absolute_error(y_val, y_pred_val)
    return(mae)

best_model = None
pipeline_xgb_4= None

def callback(study, trial):  # esto para guardar el mejor modelo
    global best_model
    if study.best_trial == trial:
        best_model = pipeline_xgb_4

In [14]:
sampler = TPESampler(seed=seed) # esto para que los resultados sean reproducibles
study = optuna.create_study(direction="minimize",sampler=sampler)
study.optimize(objective_function, timeout=300,callbacks=[callback])

[I 2023-11-17 15:05:44,866] A new study created in memory with name: no-name-f6fc28f1-b075-4d3d-a414-fdf8105dfb16


[0]	validation_0-mae:12882.05071
[1]	validation_0-mae:12309.43088
[2]	validation_0-mae:11799.50291
[3]	validation_0-mae:11362.92158
[4]	validation_0-mae:10991.28844
[5]	validation_0-mae:10662.67971
[6]	validation_0-mae:10390.67380
[7]	validation_0-mae:10155.27179
[8]	validation_0-mae:9947.42198
[9]	validation_0-mae:9778.36515
[10]	validation_0-mae:9638.12127
[11]	validation_0-mae:9511.54318
[12]	validation_0-mae:9408.14045
[13]	validation_0-mae:9322.10102
[14]	validation_0-mae:9247.90368
[15]	validation_0-mae:9190.62991
[16]	validation_0-mae:9142.81406
[17]	validation_0-mae:9095.81781
[18]	validation_0-mae:9057.82659


c:\Users\augus\anaconda3\envs\Lab1\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[19]	validation_0-mae:9024.64678
[20]	validation_0-mae:9003.42688
[21]	validation_0-mae:8982.31812
[22]	validation_0-mae:8971.60978
[23]	validation_0-mae:8958.19414
[24]	validation_0-mae:8949.26439
[25]	validation_0-mae:8940.32851
[26]	validation_0-mae:8934.41212
[27]	validation_0-mae:8928.40607
[28]	validation_0-mae:8926.17149
[29]	validation_0-mae:8923.30725
[30]	validation_0-mae:8924.23444
[31]	validation_0-mae:8921.39548
[32]	validation_0-mae:8918.02595
[33]	validation_0-mae:8915.36416
[34]	validation_0-mae:8913.03789
[35]	validation_0-mae:8911.23381
[36]	validation_0-mae:8907.30634
[37]	validation_0-mae:8905.31425
[38]	validation_0-mae:8897.63911
[39]	validation_0-mae:8895.08498
[40]	validation_0-mae:8893.06877
[41]	validation_0-mae:8893.91659
[42]	validation_0-mae:8897.83739
[43]	validation_0-mae:8895.26014
[44]	validation_0-mae:8895.68970
[45]	validation_0-mae:8898.41250
[46]	validation_0-mae:8896.85321
[47]	validation_0-mae:8897.96661
[48]	validation_0-mae:8898.60682
[49]	valid


[109]	validation_0-mae:8988.39202
[110]	validation_0-mae:8986.77792
[111]	validation_0-mae:8986.16129
[112]	validation_0-mae:8986.00439
[113]	validation_0-mae:8988.78083
[114]	validation_0-mae:8991.52031
[115]	validation_0-mae:8996.13489
[116]	validation_0-mae:8996.80540
[117]	validation_0-mae:8997.43474
[118]	validation_0-mae:9000.88427
[119]	validation_0-mae:9000.66500
[120]	validation_0-mae:9004.95504
[121]	validation_0-mae:9005.54802
[122]	validation_0-mae:9008.57527
[123]	validation_0-mae:9008.64896
[124]	validation_0-mae:9012.24168
[125]	validation_0-mae:9010.29129
[126]	validation_0-mae:9015.42890
[127]	validation_0-mae:9019.48797
[128]	validation_0-mae:9017.62423
[129]	validation_0-mae:9017.32573
[130]	validation_0-mae:9019.70875
[131]	validation_0-mae:9021.47110
[132]	validation_0-mae:9027.99645
[133]	validation_0-mae:9029.71242
[134]	validation_0-mae:9029.92957
[135]	validation_0-mae:9031.05052
[136]	validation_0-mae:9035.21926
[137]	validation_0-mae:9037.98510
[138]	validat

...

[I 2023-11-17 15:10:42,771] Trial 229 finished with value: 2173.2370525180895 and parameters: {'max_depth': 10, 'learning_rate': 0.09989189447212822, 'n_estimators': 219, 'max_leaves': 93, 'min_child_weight': 4, 'reg_alpha': 0.19397347942690746, 'reg_lambda': 0.25127751689609634, 'min_frequency': 0.03205249822403314}. Best is trial 175 with value: 2012.747162233903.
c:\Users\augus\anaconda3\envs\Lab1\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-11-17 15:10:43,313] Trial 230 pruned. Trial was pruned at iteration 0.


[0]	validation_0-mae:12627.19341
[1]	validation_0-mae:11835.92979
[2]	validation_0-mae:11126.86659
[3]	validation_0-mae:10501.42027
[4]	validation_0-mae:9901.92519
[5]	validation_0-mae:9357.22641
[6]	validation_0-mae:8879.01299
[7]	validation_0-mae:8436.89964
[8]	validation_0-mae:8028.13762
[9]	validation_0-mae:7674.39587
[10]	validation_0-mae:7336.58225
[11]	validation_0-mae:6991.67799


c:\Users\augus\anaconda3\envs\Lab1\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[12]	validation_0-mae:6687.68659
[13]	validation_0-mae:6394.27531
[14]	validation_0-mae:6158.30191
[15]	validation_0-mae:5910.66914
[16]	validation_0-mae:5703.14690
[17]	validation_0-mae:5502.66084
[18]	validation_0-mae:5298.25274
[19]	validation_0-mae:5118.94524
[20]	validation_0-mae:4956.97442
[21]	validation_0-mae:4807.90303
[22]	validation_0-mae:4684.55190
[23]	validation_0-mae:4578.07793
[24]	validation_0-mae:4447.81553
[25]	validation_0-mae:4315.80898
[26]	validation_0-mae:4197.32384
[27]	validation_0-mae:4093.73337
[28]	validation_0-mae:4000.22100
[29]	validation_0-mae:3902.95535
[30]	validation_0-mae:3824.15476
[31]	validation_0-mae:3747.39932
[32]	validation_0-mae:3661.40910
[33]	validation_0-mae:3612.00039
[34]	validation_0-mae:3552.89956
[35]	validation_0-mae:3479.71943
[36]	validation_0-mae:3427.62025
[37]	validation_0-mae:3373.29675
[38]	validation_0-mae:3308.80977
[39]	validation_0-mae:3264.91460
[40]	validation_0-mae:3234.69315
[41]	validation_0-mae:3196.42015
[42]	valid

[I 2023-11-17 15:10:47,518] Trial 231 finished with value: 2111.531519373951 and parameters: {'max_depth': 10, 'learning_rate': 0.09773303760384446, 'n_estimators': 293, 'max_leaves': 93, 'min_child_weight': 4, 'reg_alpha': 0.4610490929062261, 'reg_lambda': 0.25771597597706, 'min_frequency': 0.05992553039041924}. Best is trial 175 with value: 2012.747162233903.


In [15]:
study.best_value

2012.747162233903

El error aca es ligeramente mayor que en el caso anterior, sin prunning, sin embargo el error es similar, por lo que se peude decir que ambos modelos llevan a resultados similares.  
La subida en el error se puede deber a que alguna de las ramas que se prunaron, podrian haber sido las que finalmente den mejore resultados.

In [16]:
study.best_params

{'max_depth': 10,
 'learning_rate': 0.09740196884473817,
 'n_estimators': 632,
 'max_leaves': 93,
 'min_child_weight': 3,
 'reg_alpha': 0.44436608875704947,
 'reg_lambda': 0.1480581484934142,
 'min_frequency': 0.017460855041036696}

In [17]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_max_depth,params_max_leaves,params_min_child_weight,params_min_frequency,params_n_estimators,params_reg_alpha,params_reg_lambda,state
0,0,9737.152903,2023-11-17 15:05:44.869587,2023-11-17 15:05:51.813538,0 days 00:00:06.943951,0.095121,5,60,1,0.866176,746,0.155995,0.058084,COMPLETE
1,1,5911.123439,2023-11-17 15:05:51.825516,2023-11-17 15:05:52.682894,0 days 00:00:00.857378,0.071099,7,97,5,0.183405,69,0.212339,0.181825,COMPLETE
2,2,9061.430246,2023-11-17 15:05:52.689642,2023-11-17 15:05:56.047389,0 days 00:00:03.357747,0.052951,5,29,4,0.366362,460,0.139494,0.292145,COMPLETE
3,3,6030.698483,2023-11-17 15:05:56.054139,2023-11-17 15:05:58.689280,0 days 00:00:02.635141,0.078732,6,51,3,0.170524,239,0.046450,0.607545,COMPLETE
4,4,8995.859048,2023-11-17 15:05:58.692473,2023-11-17 15:06:07.097514,0 days 00:00:08.405041,0.094940,3,81,2,0.440152,968,0.097672,0.684233,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,227,3771.266771,2023-11-17 15:10:39.158909,2023-11-17 15:10:39.702618,0 days 00:00:00.543709,0.098289,10,91,3,0.016778,251,0.371456,0.179442,PRUNED
228,228,3085.003938,2023-11-17 15:10:39.707645,2023-11-17 15:10:40.680789,0 days 00:00:00.973144,0.096050,10,97,3,0.002295,654,0.679653,0.012392,PRUNED
229,229,2173.237053,2023-11-17 15:10:40.685800,2023-11-17 15:10:42.766182,0 days 00:00:02.080382,0.099892,10,93,4,0.032052,219,0.193973,0.251278,COMPLETE
230,230,12676.658330,2023-11-17 15:10:42.775708,2023-11-17 15:10:43.312023,0 days 00:00:00.536315,0.099965,10,44,4,0.031148,230,0.244168,0.276696,PRUNED


Hay 232 trials

In [18]:
model=best_model  # guarda el mejor modelo
with open('model_xgb_4.pkl', 'wb') as file:
    pickle.dump(model, file)

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [19]:
# Inserte su código acá
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances

In [20]:
plot_optimization_history(study)

In [21]:
plot_parallel_coordinate(study)

In [22]:
plot_param_importances(study)

- Se puede ver que a partir del trial 15 se observan mejoras notables en los resultados.  
- Se puede observar que en general los mejores trial, se obtienen en el rango de los mismos hiperparametros. Aunque hay algunos hiperparametros ue podrian tener dos valores que generen buenos resultados, por ejemplo: "max_leaves".  
- Los hiperparámetros de mayor importancia para el modelo son: min_frequency, max_depth y reg_alpha.

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento? 

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [25]:
data = {'Modelo': ['Modelo 1', 'Modelo 2', 'Modelo 3',"Modelo 4","Modelo 5"],
        'MAE': [13545.190961454393, 2458.650460262427, 2580.049121708198,1973.655046337563
,2012.747162233903]}
df = pd.DataFrame(data)
df

,Modelo,MAE
0,Modelo 1,13545.190961
1,Modelo 2,2458.650460
2,Modelo 3,2580.049122
3,Modelo 4,1973.655046
4,Modelo 5,2012.747162


El mejor rendimiento lo obtiene el modelo 4, que corresponde al modelo con optimización de hiperparametros y sin prunning. Aunque el modelo 5 obtiene resultados similares.

In [26]:
with open("model_xgb_3.pkl", 'rb') as file:  # carga el mejor modelo 
    model = pickle.load(file)
y_pred_test=model.predict(X_test)  # se predice el conjunto test
mae=mean_absolute_error(y_test, y_pred_test)
mae

1920.653645615156

Se observa que en el conjunto test hay una leve mejora en la métrica del MAE, lo cual en general no debería ser así, pero se puede deber a que este conjunto podría estar formado en mayoría con los datos que mejor logra predecir el modelo. Sin embargo las métricas son similares, por lo que se puede decir que el modelo no esta sobreajustado.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>